<a href="https://colab.research.google.com/github/abubogale342/final_project/blob/master/thesis_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ratings.csv')

In [3]:
ratings.size

5157325

In [4]:
len(ratings.user_id.unique()), len(ratings.meal_id.unique()), len(ratings.hotel_id.unique())

(27694, 362, 4)

In [0]:
ratings.user_id = ratings.user_id.astype('category').cat.codes.values
ratings.meal_id = ratings.meal_id.astype('category').cat.codes.values
ratings.hotel_id = ratings.hotel_id.astype('category').cat.codes.values

In [6]:
ratings.head()

,Unnamed: 0,user_id,meal_id,rating,hotel_id
0,0,0,253,5,2
1,1,1,255,5,2
2,2,1,25,4,0
3,3,1,307,3,3
4,4,1,32,4,0


In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, test_size=0.2)

In [8]:
train.head()

,Unnamed: 0,user_id,meal_id,rating,hotel_id
429841,429841,13617,70,5,0
223745,223745,8606,134,5,1
137855,137855,6505,259,3,2
507046,507046,15532,117,4,1
1018227,1018227,27426,69,4,0


In [0]:
test.head()
y_true = test.rating

In [10]:
import keras
n_latent_factors_user = 24
n_latent_factors_meal = 10
n_latent_factors_hotel = 4
n_latent_factors_mf = 3
n_users, n_meals, n_hotels = len(ratings.user_id.unique()), len(ratings.meal_id.unique()), len(ratings.hotel_id.unique())

Using TensorFlow backend.


In [11]:
user_input = keras.layers.Input(shape=[1],name='user')
user_embedding_mlp = keras.layers.Embedding(n_users + 1, n_latent_factors_user, name='user-Embedding-MLP')(user_input)
user_vec_mlp = keras.layers.Flatten(name='FlattenUsers-MLP')(user_embedding_mlp)
user_vec_mlp = keras.layers.Dropout(0.2)(user_vec_mlp)

user_embedding_mf = keras.layers.Embedding(n_users + 1, n_latent_factors_mf, name='user-Embedding-MF')(user_input)
user_vec_mf = keras.layers.Flatten(name='FlattenUsers-MF')(user_embedding_mf)
user_vec_mf = keras.layers.Dropout(0.2)(user_vec_mf)

W0701 23:31:21.396380 139732325050240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0701 23:31:21.463874 139732325050240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0701 23:31:21.503048 139732325050240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0701 23:31:21.504619 139732325050240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0701 23:31:21.519992 

In [0]:
meal_input = keras.layers.Input(shape=[1],name='meal')
meal_embedding_mlp = keras.layers.Embedding(n_meals + 1, n_latent_factors_meal, name='meal-Embedding-MLP')(meal_input)
meal_vec_mlp = keras.layers.Flatten(name='FlattenMeal-MLP')(meal_embedding_mlp)
meal_vec_mlp = keras.layers.Dropout(0.2)(meal_vec_mlp)

meal_embedding_mf = keras.layers.Embedding(n_meals + 1, n_latent_factors_mf, name='meal-Embedding-MF')(meal_input)
meal_vec_mf = keras.layers.Flatten(name='FlattenMeal-MF')(meal_embedding_mf)
meal_vec_mf = keras.layers.Dropout(0.2)(meal_vec_mf)

In [0]:
hotel_input = keras.layers.Input(shape=[1],name='hotel')
hotel_embedding_mlp = keras.layers.Embedding(n_hotels + 1, n_latent_factors_hotel, name='hotel-Embedding-MLP')(hotel_input)
hotel_vec_mlp = keras.layers.Flatten(name='FlattenHotels-MLP')(hotel_embedding_mlp)
hotel_vec_mlp = keras.layers.Dropout(0.2)(hotel_vec_mlp)

hotel_embedding_mf = keras.layers.Embedding(n_hotels + 1, n_latent_factors_mf, name='hotel-Embedding-MF')(hotel_input)
hotel_vec_mf = keras.layers.Flatten(name='FlattenHotels-MF')(hotel_embedding_mf)
hotel_vec_mf = keras.layers.Dropout(0.2)(hotel_vec_mf)

In [0]:
concat = keras.layers.merge.Concatenate(name='concat')([user_vec_mlp,meal_vec_mlp,hotel_vec_mlp])
pred_mf = keras.layers.merge.Multiply(name='mul')([user_vec_mf,meal_vec_mf,hotel_vec_mf])

In [0]:
concat_dropout = keras.layers.Dropout(0.2)(concat)

In [0]:
dense = keras.layers.Dense(200,name='FullyConnected')(concat_dropout)
dense_batch = keras.layers.BatchNormalization(name='Batch')(dense)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout-1')(dense_batch)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(dropout_1)
dense_batch_2 = keras.layers.BatchNormalization(name='Batch-2')(dense_2)
dropout_2 = keras.layers.Dropout(0.2,name='Dropout-2')(dense_batch_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dropout_2)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)
pred_mlp = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)
combine_mlp_mf = keras.layers.merge.Concatenate(name='concat2')([pred_mf, pred_mlp])
result_combine = keras.layers.Dense(100,name='Combine-MF-MLP')(combine_mlp_mf)
deep_combine = keras.layers.Dense(100,name='FullyConnected-4')(result_combine)
result = keras.layers.Dense(1,name='Prediction')(deep_combine)
model = keras.Model([user_input, meal_input, hotel_input], result)
opt = keras.optimizers.Adam(lr =0.01)
model.compile(optimizer='adam',loss= 'mean_absolute_error')

In [23]:
history = model.fit([train.user_id, train.meal_id,train.hotel_id], train.rating, epochs=11, verbose=1, validation_split=0.1)
model_json = model.to_json()
with open("model.json","w") as json_file:
  json_file.write(model_json)
model.save_weights("model.h5")

Train on 742654 samples, validate on 82518 samples
Epoch 1/11
742654/742654 [==============================] - 330s 445us/step - loss: 0.7674 - val_loss: 0.7496
Epoch 2/11
742654/742654 [==============================] - 342s 460us/step - loss: 0.7484 - val_loss: 0.7434
Epoch 3/11
742654/742654 [==============================] - 339s 456us/step - loss: 0.7398 - val_loss: 0.7362
Epoch 4/11
742654/742654 [==============================] - 341s 459us/step - loss: 0.7335 - val_loss: 0.7355
Epoch 5/11
742654/742654 [==============================] - 339s 456us/step - loss: 0.7297 - val_loss: 0.7306
Epoch 6/11
742654/742654 [==============================] - 340s 458us/step - loss: 0.7257 - val_loss: 0.7258
Epoch 7/11
742654/742654 [==============================] - 340s 458us/step - loss: 0.7235 - val_loss: 0.7289
Epoch 8/11
742654/742654 [==============================] - 340s 457us/step - loss: 0.7214 - val_loss: 0.7253
Epoch 9/11
742654/742654 [==============================] - 338s 455u